In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F

model_name = 'flax-community/papuGaPT2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
clusters_txt = '''
piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria 
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz
mięso_wędliny wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik
'''

In [4]:
def tokenize(word):
    ids = tokenizer(word, return_tensors='pt')['input_ids'][0]
    return [tokenizer.decode(n) for n in ids]

def cos(a, b):
    return a.dot(b) / (a.dot(a) * b.dot(b)) ** 0.5

emb = model.transformer.wte.weight.detach().cpu().numpy()
N = 50257

def find_closest(word, n=5):
    tokens = tokenize(' ' + word)
    print (tokens)
    token_id = tokenizer.encode(tokens[0])[0]
    print(emb[token_id])

    score = [(cos(emb[i], emb[token_id]), tokenizer.decode(i)) for i in range(N)]
    score.sort(reverse=True)
    return score[:n]

for s, w in find_closest('kot', 10):
    print ('   ', s, f'[{w}]')

[' kot']
[-7.06907827e-03 -7.15436116e-02 -1.50365951e-02  9.86235589e-02
 -1.65906757e-01 -1.31145924e-01  1.57749072e-01 -3.10216874e-01
 -4.13822979e-01 -2.15715766e-02  2.16775388e-01 -1.66108087e-01
  1.72237799e-01 -2.31252715e-01 -2.91370034e-01  1.91028584e-02
 -3.82684320e-01 -7.75970221e-02  3.36805314e-01 -2.53982931e-01
  6.80627897e-02  4.95083556e-02  1.57559559e-01  1.67519674e-02
  7.96861649e-02 -3.30579042e-01  3.03698927e-02 -2.19892636e-01
  1.68386549e-02  7.69453272e-02 -2.85500772e-02 -2.33997554e-02
  2.56195702e-02  1.15822833e-02 -6.97415695e-02  1.51440918e-01
  1.50915861e-01  1.37614116e-01 -1.42565325e-01  7.39018992e-02
 -1.54027328e-01  2.88279235e-01 -9.05700326e-02  1.50334328e-01
  4.20011915e-02 -2.64104661e-02 -4.93707918e-02 -3.34172398e-02
  3.00179003e-04  8.16206187e-02  8.51697847e-02 -6.26541674e-02
  2.52383947e-01 -1.87615588e-01  2.03764066e-01  1.94525793e-02
  6.95529580e-02 -3.78212720e-01 -1.22890368e-01 -1.74016319e-02
 -7.64590278e-02

In [ ]:
import random
import numpy as np

def simplestEmbeddingStr(word):
    tokens = tokenize(' ' + word)
    token_id = tokenizer.encode(tokens[0])[0]
    e = emb[token_id]
    return ' '.join(str(x) for x in e)

def sumEmbeddingStr(word):
    word = ' ' + word
    tokens = [tokenizer.decode(n) for n in tokenizer(word, return_tensors='pt')['input_ids'][0]]
    # weights = [1.0/x for x in range(1, len(tokens) + 1)]
    weights = np.exp(-np.arange(len(tokens), dtype=float) * 0.5)
    weights = [1 for x in range(len(tokens))]
    print(tokens)
    print(weights)
    out = np.zeros(emb.shape[1])
    for w, e in zip(weights, tokens):
        out += w * emb[tokenizer.encode(e)[0]]
    return ' '.join(str(x) for x in out)

def insertTypo(word : str):
    i = random.randint(0, len(word))
    return word[:i] + chr(random.randint(97, 122)) + word[i:]

with open('word_embedings_file.txt', 'w') as f:
    for line in clusters_txt.strip().split('\n'):
        # f.write(line + '\n')
        for word in line.split()[1:]:
            typo = insertTypo(word)
            print(word, typo)
            # f.write(f'{word} {simplestEmbeddingStr(word)}\n')   
            # token_emb = [emb[tokenizer.encode(t)[0]] for t in tokens]
            # f.write(f'{word} {simplestEmbeddingStr(typo)}\n')
            # f.write(f'{word} {get_word_embeding_papuga(word)}\n')
            f.write(f'{word} {sumEmbeddingStr(word)}\n')
        # f.write('\n')


pisak pidsak
[' pisa', 'k']
[1, 1]
flamaster flamasteyr
[' fla', 'master']
[1, 1]
ołówek orłówek
[' ołów', 'ek']
[1, 1]
długopis długopeis
[' długo', 'pis']
[1, 1]
pióro apióro
[' pióro']
[1]
mysz gmysz
[' mysz']
[1]
szczur sczczur
[' szczu', 'r']
[1, 1]
chomik cyhomik
[' cho', 'mik']
[1, 1]
łasica łasicma
[' ła', 'si', 'ca']
[1, 1, 1]
kuna hkuna
[' ku', 'na']
[1, 1]
bóbr bóbjr
[' b', 'óbr']
[1, 1]
niszczyciel nilszczyciel
[' niszczy', 'ciel']
[1, 1]
lotniskowiec lotniskowieic
[' lotnisk', 'owiec']
[1, 1]
trałowiec trałozwiec
[' tra', 'łowiec']
[1, 1]
krążownik krąażownik
[' krąż', 'ownik']
[1, 1]
pancernik pancernbik
[' pancer', 'nik']
[1, 1]
fregata fsregata
[' fre', 'gata']
[1, 1]
korweta koqrweta
[' kor', 'weta']
[1, 1]
lekarz leakarz
[' lekarz']
[1]
pediatra pkediatra
[' pedia', 'tra']
[1, 1]
ginekolog gineqkolog
[' gineko', 'log']
[1, 1]
kardiolog kardiologq
[' kardio', 'log']
[1, 1]
internista internistna
[' inter', 'nista']
[1, 1]
geriatra geriatrad
[' ger', 'ia', 'tra']
[1, 1,